## Setup

(1) Install IPyKernel

```console
pip install ipykernel
```

(2) Install FlexMeasures IPython Kernel

```console
ipython -m ipykernel install --user --name=fm
```

In [ ]:
from flexmeasures.app import create
from flexmeasures.data.models.time_series import Sensor

from datetime import timedelta

from flexmeasures.data.utils import save_to_db

from sqlalchemy import select, update
import time

import timely_beliefs as tb
import pandas as pd
import numpy as np

from flexmeasures.data.services.utils import get_or_create_model
from flexmeasures import Asset, Sensor, AssetType, Account, Source
from flexmeasures.data.models.time_series import TimedBelief

app = create()

In [ ]:
ctx = app.app_context()

In [ ]:
ctx.__enter__()

In [ ]:
from flexmeasures.data.models.planning.linear_optimization import device_scheduler
import pandas as pd

COLUMNS = [
    "equals",
    "max",
    "min",
    "efficiency",
    "derivative equals",
    "derivative max",
    "derivative min",
    "derivative down efficiency",
    "derivative up efficiency",
    "stock delta",
]

In [ ]:
import numpy as np
resolution = timedelta(minutes=15)
dt = pd.date_range("2024-01-01", "2024-01-02", freq="15min", inclusive="left")

commitment_upwards_deviation_price = pd.Series([100]*len(dt), index=dt) # Consumption Price
commitment_downwards_deviation_price = pd.Series([70]*len(dt), index=dt) # Production

#commitment_downwards_deviation_price[:10] = 10
#commitment_upwards_deviation_price[:10] = 10

commitment_upwards_deviation_price += -np.arange(len(dt))*0.2
commitment_downwards_deviation_price += -np.arange(len(dt))*0.2

commitment_quantities = pd.Series([0]*len(dt), index=dt)

ems_constraints = pd.DataFrame(columns=COLUMNS, index=dt)
ems_constraints["derivative max"] = 1
ems_constraints["derivative min"] = -1

n_devices = 1

soc_at_start = 0.1

soc_min = 0.0
soc_max = 0.9

# Base device constraints
dc_main =  pd.DataFrame(columns=COLUMNS, index=dt)
dc_main["min"] = (soc_min - soc_at_start) * timedelta(hours=1) / resolution
dc_main["max"] =  (soc_max - soc_at_start) * timedelta(hours=1) / resolution

dc_main["efficiency"] = 0.99

dc_main["derivative max"] = 1
dc_main["derivative min"] = -1

dc_main["derivative down efficiency"] = 0.9
dc_main["derivative up efficiency"] = 0.9

device_constraints = []

#device_constraints.append(dc_main)

for i in range(n_devices):
    dc = dc_main.copy()
    #dc["derivative down efficiency"] += 1
    device_constraints.append(dc)

# Add some inflexible
dc =  pd.DataFrame(columns=COLUMNS, index=dt)
dc["derivative equals"] = 1
dc["derivative equals"][10:20] = -2
dc["derivative equals"][20:40] = -1.5
dc["derivative equals"][40:50] = -2
dc["derivative equals"][70:80] = -2
dc *= -1
device_constraints.append(dc)


In [ ]:
planned_power_per_device, planned_costs, results, model = device_scheduler(
    device_constraints,
    ems_constraints,
    [commitment_quantities],
    [commitment_downwards_deviation_price],
    [commitment_upwards_deviation_price],
    initial_stock = soc_at_start * (timedelta(hours=1) / resolution),
    relaxed=True,
    relaxation_cost=1009
)

In [ ]:
results.solver.termination_condition

In [ ]:
planned_costs

In [ ]:
import matplotlib.pyplot as plt
for i, power in enumerate(planned_power_per_device):
    plt.plot(power, label=i)
plt.legend()
plt.grid()
plt.tight_layout()

In [ ]:
min(sum(planned_power_per_device)), max(sum(planned_power_per_device))

In [ ]:
plt.plot(sum(planned_power_per_device))

In [ ]:
plt.plot(commitment_upwards_deviation_price, label="up")
plt.plot(commitment_downwards_deviation_price, label="down")

plt.legend()